In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import models
import learn
#from FnLearn import Postprocessing

In [ ]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

tf.enable_eager_execution(config = config)

In [ ]:
item_names = ['삼성전자','한국전력','SK텔레콤','니케이 225','S&P 500','코스피 200']
#price_col_name = '수정주가(원)'
price_col_names = ['수정주가(원)','수정주가(원)','수정주가(원)','종가지수(포인트)','종가지수(포인트)','종가지수(포인트)']
model_name = 'LSTM'
channel = False

In [ ]:
#HyperParameter
target_types = ['trend_ratio','log_ratio']
fill_na_li = ['interpolation','zero','median']
future_days = [20,40,60]
n_timesteps = [20,40,60,80,100,120]
time_interval = 1
n_units = [300,500,800]
batch_size = 32
learning_rate = 0.0005
n_iteration = 10000

In [ ]:
for item_name, price_col_name in zip(item_names,price_col_names):
    for target_type in target_types:
        for fill_na in fill_na_li:
            for n_unit in n_units:
                for future_day in future_days :
                    for n_timestep in n_timesteps:
                        train_input, train_output, test_input, test_output,test_price = FnLearn.create_learning_data(item_name, n_timestep, future_day,time_interval, price_col_name, target_type, fill_na, channel)
                        n_input = train_input.shape[2]

                        model = models.LSTM(n_timestep,n_input,n_unit)
                        #keras.utils.plot_model(model, model_name+'_model_with_shape_info.png', show_shapes=True)

                        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

                        for iteration in range(n_iteration):
                            batch_input, batch_output = learn.next_random_batch(train_x, train_x, batch_size)

                            gradients = models.gradient(model, 'mean_square', batch_input, batch_output)
                            optimizer.apply_gradients(zip(gradients, model.variables))

                            if iteration %100 == 0:
                                loss = models.loss_function(model, batch_input, batch_output)
                                train_MSE = models.evaluate(model, batch_input, batch_output)
                                test_MSE =  models.evaluate(model, test_input[:100], test_output[:100])

                                print('iteration :', iteration, ' loss =', loss.numpy(), ' train MSE =', train_MSE.numpy(),' test MSE =', test_MSE.numpy())

                        prediction = FnLearn.predict_batch_test(model, test_input,100)

                        result = Postprocessing(prediction,test_output, n_timestep,future_day)
                        result.extract_last_output()
                        result.ratio_to_price(test_price,target_type)
                        result.result_df()
                        result.calculation_rise_fall()
                        result.evaluation()
                        result.save_result(model_name,item_name,n_unit,target_type,fill_na)
                        result.save_visualization()
                        result.save_model_architecture(model,model_name)
                        result.save_model(model)